### 【問題1】中心点の初期値を決める

### 【問題2】SSEを求める関数の作成

### 【問題3】クラスタへの割り当て

### 【問題4】中心点の移動

### 【問題5】繰り返し

### 【問題6】異なる初期値で計算

### 【問題7】推定

### 【問題8】エルボー法の実装

### 【問題9】（アドバンス課題）シルエット図

### 【問題10】クラスタ数kの選択

### 【問題11】既知のグループとの比較

### 【問題12】卸売業者にとって有益な情報

### 【問題13】（アドバンス課題）他の手法の調査

### 【問題14】（アドバンス課題）t-SNEとDBSCANの利用